<a href="https://colab.research.google.com/github/Tom-Jung/testttttt/blob/main/07_%EC%9B%B9%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_%EB%89%B4%EC%8A%A4(%EC%88%98%EC%97%85%EC%A0%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# [데이터 수집] 네이버 뉴스

- BeautifulSoup, Selenium을 적용하여 데이터수집을 한다.

### 스크래핑

- [뉴스 목록 url](https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2024.09.01&de=2024.09.15&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20240901to20240915&is_sug_officeid=0&office_category=0&service_area=0"
)
- 검색어: 삼성전자
- 기간: 2024년 9월 1일 ~ 9월 15일
- 스크롤을 내리는 방식으로 새로운 뉴스 업데이트

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/kdi/202410실습자료')
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/kdi/202410실습자료'

In [35]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import requests
import time
import re

# URL 설정
url = 'http://search.naver.com/search.naver?sm=tab_hty.top&where=news&ssc=tab.news.all&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&oquery=s&tqi=ixbuIlqVOsCssvYvoHZssssst2R-252050&nso=so%3Ar%2Cp%3Afrom20240901to20240915&de=2024.09.15&ds=2024.09.01&mynews=0&office_category=0&office_section_code=0&office_type=0&pd=3&photo=0&service_area=0&sort=0'


# 헤더 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "DNT": "1",
}

In [ ]:
df = pd.read_excel('20241017-1741_종목_005380.xlsx')
df

In [14]:
# 결측치 탐지
df.isna().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   날짜          21 non-null     object 
 1   제목          21 non-null     object 
 2   글쓴이         21 non-null     object 
 3   조회          21 non-null     object 
 4   공감          21 non-null     object 
 5   비공감         21 non-null     object 
 6   Unnamed: 6  0 non-null      float64
dtypes: float64(1), object(6)
memory usage: 1.5+ KB


In [15]:
df.dropna()

,날짜,제목,글쓴이,조회,공감,비공감,Unnamed: 6


In [19]:
# df.dropna(how='all')   # 별도설정하지 않으면 행기준 제거
df.dropna(how='all',axis=1)   # 별도설정하지 않으면 행기준 제거

,날짜,제목,글쓴이,조회,공감,비공감
0,NaN,NaN,NaN,NaN,NaN,NaN
1,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정
2,NaN,NaN,NaN,NaN,NaN,NaN
3,2024.10.17 17:10,클린봇이 이용자 보호를 위해 숨긴 게시글입니다. [2],laro****,69,1,2
4,2024.10.17 17:09,■ 인도청약률 166% 달성 [7],tt******,360,6,3
5,2024.10.17 17:05,환율 1369원이네요,veni****,75,0,0
6,2024.10.17 17:05,변속기 파업 때문에,aksk****,101,1,1
7,2024.10.17 17:04,### 10월 24일 2시 실적발표 공시... [3],leeb****,244,3,3
8,NaN,NaN,NaN,NaN,NaN,NaN
9,2024.10.17 17:04,시간외 주가는 ㅋㅋㅋ [1],infi****,211,1,3


In [ ]:
df.dropna(how='any')
df.dropna(how='any',axis=0)
df.dropna(how='any',axis=1)

In [26]:
df.dropna(how='all',axis=1,inplace=True)
df

,날짜,제목,글쓴이,조회,공감,비공감
0,NaN,NaN,NaN,NaN,NaN,NaN
1,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정
2,NaN,NaN,NaN,NaN,NaN,NaN
3,2024.10.17 17:10,클린봇이 이용자 보호를 위해 숨긴 게시글입니다. [2],laro****,69,1,2
4,2024.10.17 17:09,■ 인도청약률 166% 달성 [7],tt******,360,6,3
5,2024.10.17 17:05,환율 1369원이네요,veni****,75,0,0
6,2024.10.17 17:05,변속기 파업 때문에,aksk****,101,1,1
7,2024.10.17 17:04,### 10월 24일 2시 실적발표 공시... [3],leeb****,244,3,3
8,NaN,NaN,NaN,NaN,NaN,NaN
9,2024.10.17 17:04,시간외 주가는 ㅋㅋㅋ [1],infi****,211,1,3


In [27]:
df.drop(columns=['조회'])

,날짜,제목,글쓴이,공감,비공감
0,NaN,NaN,NaN,NaN,NaN
1,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정,클린봇으로 착한게시글만 모아보세요! 설정
2,NaN,NaN,NaN,NaN,NaN
3,2024.10.17 17:10,클린봇이 이용자 보호를 위해 숨긴 게시글입니다. [2],laro****,1,2
4,2024.10.17 17:09,■ 인도청약률 166% 달성 [7],tt******,6,3
5,2024.10.17 17:05,환율 1369원이네요,veni****,0,0
6,2024.10.17 17:05,변속기 파업 때문에,aksk****,1,1
7,2024.10.17 17:04,### 10월 24일 2시 실적발표 공시... [3],leeb****,3,3
8,NaN,NaN,NaN,NaN,NaN
9,2024.10.17 17:04,시간외 주가는 ㅋㅋㅋ [1],infi****,1,3


In [29]:
df.dropna(how='all',inplace=True) # 행기준 nan인거 제거

In [34]:
# 제목에 대해 클린봇 문자열이 있는 경우 제거
df['제목'].str.contains('클린봇')
df[~df['제목'].str.contains('클린봇')]

,날짜,제목,글쓴이,조회,공감,비공감
4,2024.10.17 17:09,■ 인도청약률 166% 달성 [7],tt******,360,6,3
5,2024.10.17 17:05,환율 1369원이네요,veni****,75,0,0
6,2024.10.17 17:05,변속기 파업 때문에,aksk****,101,1,1
7,2024.10.17 17:04,### 10월 24일 2시 실적발표 공시... [3],leeb****,244,3,3
9,2024.10.17 17:04,시간외 주가는 ㅋㅋㅋ [1],infi****,211,1,3
10,2024.10.17 17:03,내일도 돈급하고 견딜수 없으면 시장가로 ... [1],rkdg****,92,0,1
11,2024.10.17 16:58,■ 인도청약률 141% 달성 [4],tt******,393,7,2
12,2024.10.17 17:01,■ 인도청약률 141% 달성 [1],to1t****,140,1,2
13,2024.10.17 16:53,인도청약? [2],infi****,239,2,8
15,2024.10.17 17:01,인도청약?,veni****,39,0,1


In [36]:
# requests 로 HTTP GET 요청
response = requests.get(url, headers=headers)

In [37]:
# 응답코드 확인 (200번대)
response.status_code

200

In [38]:
# BeautifulSoup 객체 생성
soup = BeautifulSoup(response.text, 'html.parser')

In [39]:
# <h1> 태그 찾기
soup.find('h1')

<h1 class="logo"><a class="link" href="https://www.naver.com" onclick="return goOtherCR(this, 'a=sta.naver&amp;r=&amp;i=&amp;u='+urlencode(this.href));"><i class="spnew2 ico_logo">NAVER</i></a></h1>

In [ ]:
# 뉴스 기사 가져오기 - ul태그
# soup.find('div',class_='group_news')
soup.find('ul', class_='list_news _infinite_list')

In [47]:
# 뉴스 기사 가져오기 - a태그
soup.find_all('a','news_tit')

[<a class="news_tit" href="https://magazine.hankyung.com/business/article/202409150211b" onclick="return goOtherCR(this, 'a=nws*j.tit&amp;r=1&amp;i=88000106_000000000000000000079831&amp;g=050.0000079831&amp;u='+urlencode(this.href));" target="_blank" title="삼성전자·LG전자에 완전 밀렸다...'가전 왕국' 일본의 추락"><mark>삼성전자</mark>·LG전자에 완전 밀렸다...'가전 왕국' 일본의 추락</a>,
 <a class="news_tit" href="https://www.yna.co.kr/view/AKR20240913079700003?input=1195m" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=2&amp;i=880000D8_000000000000000014930926&amp;g=001.0014930926&amp;u='+urlencode(this.href));" target="_blank" title="Arm, 삼성전자 사장 출신 손영권 이사 선임">Arm, <mark>삼성전자</mark> 사장 출신 손영권 이사 선임</a>,
 <a class="news_tit" href="https://www.news1.kr/industry/electronics/5542625" onclick="return goOtherCR(this, 'a=nws*e.tit&amp;r=5&amp;i=08138263_000000000000000007791694&amp;g=421.0007791694&amp;u='+urlencode(this.href));" target="_blank" title="삼성전자 HBM3E 8단 공급 시작…AI 칩 '삼성의 시간' 온다"><mark>삼성전자</mark> HBM3E 8단 공급 시작…AI 칩 '삼

### BeautifulSoup .get()

- `BS.get()` : 특정 태그를 명시한 후, 각 태그에 대해 `.get()` 에 들어간 속성값을 가져온다.

- 예) BS.get('href') : href의 속성값을 가져옴

In [49]:
# 뉴스 기사 가져오기
# 뉴스목록 = soup.find_all('a','news_tit')

In [54]:
# 뉴스 제목 가져오기
뉴스목록 = soup.find_all('a','news_tit')
for i in 뉴스목록:
    print(i.get('title'))

삼성전자·LG전자에 완전 밀렸다...'가전 왕국' 일본의 추락
Arm, 삼성전자 사장 출신 손영권 이사 선임
삼성전자 HBM3E 8단 공급 시작…AI 칩 '삼성의 시간' 온다
삼성전자 박학규 사장, 자사주 4억원어치 매입
세계시장 휩쓰는 中 로봇청소기…보안·위생 앞세워 추격 나선 삼성·LG전자
'밸류다운' 삼성전자...이재용 리더십이 안 보인다?
삼성전자, "구매 전 과정에 만족"…고객경험지수1위
삼성전자, 디지털고객경험지수 종합가전 2년째 1위
자사주 ‘줍줍’하는 임원들…삼성전자 ‘최다’
삼성·LG전자, 국제 디자인 공모전 ‘IDEA 2024′ 주요 상 휩쓸어


In [46]:
# 뉴스 링크 가져오기
soup.find_all('a','news_wrap api_ani_send')

[]

### BeautifulSoup .find_next_sibling()

- `BS.find_next_sibling()` : 현재 태그의 다음 형제 태그를 찾는다.

- `find()` 와 유사하나 웹페이지 전체에서 특정 태그를 찾는 것이 아니라 특정 태그 범위에서 검색하므로 속도가 빠르고, 원하는 결과를 얻기 쉽다.

- 예) BS.find_next_sibling('div', 'title')

In [56]:
# 뉴스 본문 일부 가져오기
뉴스목록 = soup.find_all('a','news_tit')
for i in 뉴스목록:
    본문 = i.find_next_sibling('div','news_dsc')
    print(본문.text)

  한때 소니는 유력 가전기업으로서 삼성전자와 부스 면적을 놓고 자존심 싸움을 벌이는 등 IFA에서 활발히 활동해왔다. 삼성전자는 2006년 IFA에서 3900㎡ 규모의 부스를 설치했다. 그러자 소니는 2008년 5950㎡로 면적을 넓히며 '세'를 과시했다. 당시 삼성전자(3900㎡), LG전자(3000㎡) 부스를 합친 것과 맞먹는... 
  손 이사는 삼성전자에서 최고전략책임자(CSO) 사장으로서 투자와 신사업 전략을 이끌며 80억달러 규모 하만 인터내셔널 인수를 주도했다. 또 그는 하만 이사회 의장, 케이던스 이사회 멤버, 월든 카탈리스트 창립 매니징 파트너 등을 맡았다. 르네 하스 Arm의 최고경영자(CEO)는 "손 이사는 세계 유수의 반도체... 
  삼성전자(005930)가 인공지능(AI) 반도체 생태계에 합류하기 위해 추격 속도를 높이고 있다. 최근 5세대 고대역폭메모리(HBM3E) 8단 제품 출하가 시작됐다. 최대 관심사인 엔비디아 납품은 아직이지만 8단 제품 양산을 시작으로 하반기 AI 메모리 시장에서 반등의 물꼬를 틀 수 있을지 주목된다. 15일 반도체... 
  삼성전자 사장단이 책임경영 강화 차원에서 회사 주식 매입에 잇달아 나서고 있다. 13일 금융감독원 전자공시시스템에 따르면 박학규 삼성전자 경영지원실장(사장)은 전날 회사 주식 4억110만원어치를 매입했다. 한 주당 6만6850원씩 총 6000주를 사들였다. 박 사장이 보유한 삼성전자 주식은... 
  추격에 나선 삼성전자와 LG전자는 올해 보안 기술과 위생·살균 기능을 강화한 신모델을 출시하며 반격에 나서고 있다. 15일 업계에 따르면 국내 시장뿐 아니라 글로벌 시장점유율 1위를 달성한 중국 청소 가전 전문기업 로보락은 1500여명 임직원 중 절반 이상이 연구개발 직군이다. 이를 바탕으로 로보락은... 
  삼성전자 주가가 연일 미끄럼을 타고 있다. 6만원대로 추락한 주가는 아직 바닥이 보이지 않는다. 시장은 삼성전자를 불안한 시선으로 바라본다. 하반기 반도체 업황도 만만치 않은데, 

### 결과 저장

In [59]:
# 결과 저장할 빈 리스트 생성
결과 = []

# 각 뉴스 제목, 링크 본문
뉴스목록 = soup.find_all('a','news_tit')
for i in 뉴스목록:
    제목 = i.get('title')
    링크 = i.get('href')
    본문 = i.find_next_sibling('div','news_dsc').text
    결과.append({'title':제목,
              'link':링크,
              'summary':본문
              })
print(결과)


[{'title': "삼성전자·LG전자에 완전 밀렸다...'가전 왕국' 일본의 추락", 'link': 'https://magazine.hankyung.com/business/article/202409150211b', 'summary': "  한때 소니는 유력 가전기업으로서 삼성전자와 부스 면적을 놓고 자존심 싸움을 벌이는 등 IFA에서 활발히 활동해왔다. 삼성전자는 2006년 IFA에서 3900㎡ 규모의 부스를 설치했다. 그러자 소니는 2008년 5950㎡로 면적을 넓히며 '세'를 과시했다. 당시 삼성전자(3900㎡), LG전자(3000㎡) 부스를 합친 것과 맞먹는... "}, {'title': 'Arm, 삼성전자 사장 출신 손영권 이사 선임', 'link': 'https://www.yna.co.kr/view/AKR20240913079700003?input=1195m', 'summary': '  손 이사는 삼성전자에서 최고전략책임자(CSO) 사장으로서 투자와 신사업 전략을 이끌며 80억달러 규모 하만 인터내셔널 인수를 주도했다. 또 그는 하만 이사회 의장, 케이던스 이사회 멤버, 월든 카탈리스트 창립 매니징 파트너 등을 맡았다. 르네 하스 Arm의 최고경영자(CEO)는 "손 이사는 세계 유수의 반도체... '}, {'title': "삼성전자 HBM3E 8단 공급 시작…AI 칩 '삼성의 시간' 온다", 'link': 'https://www.news1.kr/industry/electronics/5542625', 'summary': '  삼성전자(005930)가 인공지능(AI) 반도체 생태계에 합류하기 위해 추격 속도를 높이고 있다. 최근 5세대 고대역폭메모리(HBM3E) 8단 제품 출하가 시작됐다. 최대 관심사인 엔비디아 납품은 아직이지만 8단 제품 양산을 시작으로 하반기 AI 메모리 시장에서 반등의 물꼬를 틀 수 있을지 주목된다. 15일 반도체... '}, {'title': '삼성전자 박학규 사장, 자사주 4억원어치 매입', 'link': 'https://www

In [60]:
# 결과를 데이터프레임으로 변환 후 출력
df = pd.DataFrame(결과)
df

,title,link,summary
0,삼성전자·LG전자에 완전 밀렸다...'가전 왕국' 일본의 추락,https://magazine.hankyung.com/business/article...,한때 소니는 유력 가전기업으로서 삼성전자와 부스 면적을 놓고 자존심 싸움을 벌이...
1,"Arm, 삼성전자 사장 출신 손영권 이사 선임",https://www.yna.co.kr/view/AKR2024091307970000...,손 이사는 삼성전자에서 최고전략책임자(CSO) 사장으로서 투자와 신사업 전략을 ...
2,삼성전자 HBM3E 8단 공급 시작…AI 칩 '삼성의 시간' 온다,https://www.news1.kr/industry/electronics/5542625,삼성전자(005930)가 인공지능(AI) 반도체 생태계에 합류하기 위해 추격 속...
3,"삼성전자 박학규 사장, 자사주 4억원어치 매입",https://www.newsis.com/view/NISX20240913_00028...,삼성전자 사장단이 책임경영 강화 차원에서 회사 주식 매입에 잇달아 나서고 있다....
4,세계시장 휩쓰는 中 로봇청소기…보안·위생 앞세워 추격 나선 삼성·LG전자,https://www.seoul.co.kr/news/economy/2024/09/1...,추격에 나선 삼성전자와 LG전자는 올해 보안 기술과 위생·살균 기능을 강화한 신...
5,'밸류다운' 삼성전자...이재용 리더십이 안 보인다?,http://news.mt.co.kr/mtview.php?no=20240913164...,삼성전자 주가가 연일 미끄럼을 타고 있다. 6만원대로 추락한 주가는 아직 바닥이...
6,"삼성전자, ""구매 전 과정에 만족""…고객경험지수1위",https://www.newsis.com/view/NISX20240913_00028...,삼성전자가 최근 한국표준협회 '디지털고객경험지수(DCXI)'에서 종합가전 부문 ...
7,"삼성전자, 디지털고객경험지수 종합가전 2년째 1위",http://www.fnnews.com/news/202409130850233021,삼성전자는 지난 5일 한국표준협회 '디지털고객경험지수(DCXI)'에서 종합가전 ...
8,자사주 ‘줍줍’하는 임원들…삼성전자 ‘최다’,https://magazine.hankyung.com/business/article...,특히 최근 52주 신저가 기록을 갈아치운 삼성전자 임원들이 자사주를 가장 많이 ...
9,"삼성·LG전자, 국제 디자인 공모전 ‘IDEA 2024′ 주요 상 휩쓸어",https://biz.chosun.com/it-science/ict/2024/09/...,"삼성전자, 금상 2개 등 45개 수상… LG전자 23개 상 받아 삼성전자와 LG..."


### 코드 정리


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import requests
import time
import re

# URL 설정
url = 'http://search.naver.com/search.naver?sm=tab_hty.top&where=news&ssc=tab.news.all&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&oquery=s&tqi=ixbuIlqVOsCssvYvoHZssssst2R-252050&nso=so%3Ar%2Cp%3Afrom20240901to20240915&de=2024.09.15&ds=2024.09.01&mynews=0&office_category=0&office_section_code=0&office_type=0&pd=3&photo=0&service_area=0&sort=0'


# 헤더 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "DNT": "1",
}

# request 로 HTTP GET 요청
response = requests.get(url, headers=headers)
print(response.status_code)

# 결과 저장할 빈 리스트 생성
results = []

# 각 뉴스 제목, 링크 본문
for news in news_list:
    title = news.get('title')
    link = news.get('href')
    summary = news.find_next_sibling('div', class_='news_dsc').get_text()
    results.append({
        'title': title,
        'link': link,
        'summary': summary
    })


# 결과를 데이터프레임으로 변환 후 출력
df = pd.DataFrame(results)
df

### 구글드라이브 연동 및 작업 경로 설정
- `import os`
- `os.chdir()`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/kbi/202410실습자료')
os.getcwd()

In [ ]:
# 저장


---
# (참고) Selenium 과 Schedule 적용
- 동적 웹스크래핑 기능

### 필요 라이브러리 설치

- 필요 라이브러리 설치

In [61]:
# 셀레니움 및 크롬드라이버 설치 (시간 조금 소요)
!apt-get update
!apt install -y chromium-chromedriver
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,032 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,598 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jam

In [62]:
import time
import schedule
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

# 사용자 정의 함수: 페이지 스크롤 (n번 스크롤)
def scroll_down_n_times(driver, n):
    for _ in range(n):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2)  # 페이지 로드 대기

# 사용자 정의 함수: 데이터 크롤링
def crawl_news(scroll_limit=5):  # 스크롤 제한 추가
    url = "https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=1&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Add%2Cp%3Aall&is_sug_officeid=0&office_category=0&service_area=0"

    # 웹 드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    results = []
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 스크롤을 scroll_limit 횟수만큼 수행
    scroll_down_n_times(driver, scroll_limit)

    # 뉴스 기사 정보 추출
    news_list = driver.find_elements(By.CLASS_NAME, 'news_tit')
    for news in news_list:
        title = news.get_attribute('title')
        link = news.get_attribute('href')
        summary_tag = news.find_element(By.XPATH, ".//following-sibling::div[@class='news_dsc']")
        summary = summary_tag.text
        results.append({
            'Title': title,
            'Link': link,
            'Summary': summary,
        })

    driver.quit()

    # 데이터프레임으로 변환
    df = pd.DataFrame(results)
    return df

# 예시 실행: 3번 스크롤 후 데이터 크롤링
df = crawl_news(scroll_limit=3)

# 기사 링크가 중복인 경우 마지막 행만 남겨두고 삭제
df_cleaned = df.drop_duplicates(subset='Link', keep='last')

print(df)

ModuleNotFoundError: No module named 'schedule'